In [1]:
import torch
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

X, y = make_classification(
    n_samples=1000,      # number of samples
    n_features=10,       # number of features
    n_informative=8,     # informative features
    n_redundant=2,       # redundant features
    n_classes=3,         # number of classes
    random_state=42
)

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

In [ ]:
import torch 
import torch.nn as nn 

class GaussianNB_PyTorch:
    def __init__(self):
        self.class_means = None
        self.class_vars = None
        self.class_priors = None
        self.classes = None

    def fit(self, X, y):
        X = torch.tensor(X, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.long)
        self.classes = torch.unique(y)
        n_features = X.shape[1]
        n_classes = len(self.classes)

        self.class_means = torch.zeros((n_classes, n_features))
        self.class_vars = torch.zeros((n_classes, n_features))
        self.class_priors = torch.zeros(n_classes)

        for idx, c in enumerate(self.classes):
            X_c = X[y == c]
            self.class_means[idx] = X_c.mean(dim=0)
            self.class_vars[idx] = X_c.var(dim=0) + 1e-6  
            self.class_priors[idx] = X_c.shape[0] / X.shape[0]

    def _gaussian_log_prob(self, X, mean, var):
        # log Gaussian probability
        return -0.5 * torch.sum(torch.log(2 * torch.pi * var) + ((X - mean)**2) / var, dim=1)

    def predict(self, X):
        X = torch.tensor(X, dtype=torch.float32)
        log_posteriors = []

        for idx, c in enumerate(self.classes):
            log_prior = torch.log(self.class_priors[idx])
            log_likelihood = self._gaussian_log_prob(X, self.class_means[idx], self.class_vars[idx])
            log_posteriors.append(log_prior + log_likelihood)

        log_posteriors = torch.stack(log_posteriors, dim=1)
        preds = torch.argmax(log_posteriors, dim=1)
        return preds

In [19]:
model = GaussianNB_PyTorch()
model.fit(X_train, y_train)

y_pred = model.predict(X_val)
y_val = y_val.reshape(-1)  # if y_val is (n_samples,1)
y_pred_np = y_pred.detach().cpu().numpy().reshape(-1)  # flatten in case it's 2D
accuracy = accuracy_score(y_val, y_pred_np)
print("Validation Accuracy:", accuracy)
print("\nClassification Report:\n", classification_report(y_val, y_pred_np))
print("\nConfusion Matrix:\n", confusion_matrix(y_val, y_pred_np))

Validation Accuracy: 0.755

Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.84      0.82        67
           1       0.69      0.76      0.72        67
           2       0.77      0.67      0.72        66

    accuracy                           0.76       200
   macro avg       0.76      0.75      0.75       200
weighted avg       0.76      0.76      0.75       200


Confusion Matrix:
 [[56  7  4]
 [ 7 51  9]
 [ 6 16 44]]
